In [3]:
import torch
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

In [27]:
train_input, train_target, train_classes, \
test_input, test_target, test_classes = \
    prologue.generate_pair_sets(N)

In [ ]:
#code from practical 4b
def leq(x,y):
    return x<=y

class Net(nn.Module):
    def __init__(self, nb_hidden):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Softmax(nb_hidden, 10)
        #add a softmax to identify the class?

    def forward(self, x_0, x_1):
        #process the first image
        x_0 = F.relu(F.max_pool2d(self.conv1(x_0), kernel_size=3, stride=3))
        x_0 = F.relu(F.max_pool2d(self.conv2(x_0), kernel_size=2, stride=2))
        x_0 = F.relu(self.fc1(x_0.view(-1, 256)))
        x_0 = self.fc2(x_0)
        #process the second image
        x_1 = F.relu(F.max_pool2d(self.conv1(x_1), kernel_size=3, stride=3))
        x_1 = F.relu(F.max_pool2d(self.conv2(x_1), kernel_size=2, stride=2))
        x_1 = F.relu(self.fc1(x_1.view(-1, 256)))
        x_1 = self.fc2(x_1)
        return x_0, x_1
    
def train_model(model, train_input, train_target, mini_batch_size, nb_epochs = 25):
    #binary output
    criterion = nn.CrossEntropyLoss()
    eta = 1e-1
    
    for e in range(nb_epochs):
        acc_loss = 0
        # We do this with mini-batches
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size)[:,0,:].unsqueeze(1),
                           train_input.narrow(0, b, mini_batch_size)[:,1,:].unsqueeze(1))
            #output=[list(output[0]), list(output[1])]
            print(output[0])
            output = [leq(x,y) for x, y in output]

            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            acc_loss = acc_loss + loss.item()

            model.zero_grad()
            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= eta * p.grad

            print(e, acc_loss)
        
def compute_nb_errors(model, input, target, mini_batch_size):
    nb_error = 0
    
    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size)[:,0,:].unsqueeze(1),
                       input.narrow(0, b, mini_batch_size)[:,1,:].unsqueeze(1))
        print(output.shape)
        pred_class = leq(1,0)

        for k in range(mini_batch_size):
            if target[b+k, predicted_class[k]] <=0:
                nb_error += 1
                
    return nb_error

# weight sharing -> siamese network 

In [ ]:
N=200
epochs=25

In [ ]:
model = Net(20)

In [92]:
mini_batch_size = 100

train_model(model, train_input, train_target, mini_batch_size)
nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                  nb_test_errors, test_input.size(0)))

[tensor([ -0.4208,  -5.4709,   6.0401, -12.2032,  12.7707,  12.4463,  -4.4911,
         -5.9758,   5.5035,   3.2582], grad_fn=<UnbindBackward>), tensor([ 0.2838, -2.8048,  8.0472, -6.3075, 14.8049, 10.2375, -1.7444, -9.0876,
        -2.1644,  8.5987], grad_fn=<UnbindBackward>), tensor([  3.0827,  -5.7433,   9.4372, -13.1287,  17.4882,  11.9774,  -6.7164,
         -8.0132,   1.6518,   7.6897], grad_fn=<UnbindBackward>), tensor([ -0.0731,  -0.6275,  11.0104,  -9.2722,  16.4801,  10.4996,  -0.7556,
        -11.2877,  -3.1023,   5.1997], grad_fn=<UnbindBackward>), tensor([  3.5514,  -3.7896,   7.5548, -11.8639,  18.4393,  13.4679,  -5.2337,
         -8.2458,   1.0553,   7.4655], grad_fn=<UnbindBackward>), tensor([  3.2798,  -5.7587,   9.4140, -14.0973,  14.8221,  12.4307,  -5.8188,
         -8.0522,   4.2987,   2.9037], grad_fn=<UnbindBackward>), tensor([ -0.3370,  -2.5119,   5.2385, -10.1540,  17.2144,  14.5916,  -3.2886,
         -5.9767,   2.4920,   5.8819], grad_fn=<UnbindBackward>), t

ValueError: too many values to unpack (expected 2)